In [ ]:
# 1. Environment Setup
# --------------------
print("Phase 1: Environment Setup")
print("==========================")

try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except Exception as e:
    print("Google Drive mount not required or failed:", e)

import subprocess
print("Installing/Updating Ultralytics and other necessary packages...")
try:
    # Ensure ultralytics is up-to-date for latest QAT features if any
    subprocess.run(["pip", "install", "--upgrade", "ultralytics", "torchvision", "torchaudio"], check=True)
    subprocess.run(["pip", "install", "roboflow", "tqdm", "pandas", "seaborn", "matplotlib", "pyyaml", "scipy"], check=True) # Added scipy for k-means
    print("Packages installed/updated successfully.")
except subprocess.CalledProcessError as e:
    print(f"Error installing packages: {e}")
    raise

Phase 1: Environment Setup
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.
Installing/Updating Ultralytics and other necessary packages...
Packages installed/updated successfully.


In [ ]:
!ip install torch torchvision torchaudio ultralytics tqdm

Object "install" is unknown, try "ip help".


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import yaml
import numpy as np
from scipy.cluster.vq import kmeans
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ultralytics import YOLO
from torch.ao.quantization import QuantStub, DeQuantStub
from ultralytics.nn.modules import Conv, C2f, Detect, Concat # Import building blocks
from tqdm import tqdm as tqdm_iterator # Renamed to avoid conflict with YOLO's internal tqdm
from IPython.display import Image, display # For displaying plots in Colab
from ultralytics.utils.plotting import plot_results as ultralytics_plot_results # Alias for clarity
from torch.ao.quantization import QuantStub, DeQuantStub


print("All initial imports and setup completed.\n")

All initial imports and setup completed.



In [ ]:
# Load YOLO12n model
model = YOLO('yolo12n.pt')

# Create dummy input tensor
dummy_input = torch.randn(1, 3, 640, 640)

# Extract backbone layers (all but the last Detect layer)
layers = model.model.model[:-1]

# Forward pass through the backbone, correctly handling Concat layers
outputs = []
x = dummy_input
for layer in layers:
    if layer.f != -1:
        x = layer([outputs[i] for i in layer.f])
    else:
        x = layer(x)
    outputs.append(x)

# Last three outputs are backbone outputs (feature maps)
p3, p4, p5 = outputs[-3], outputs[-2], outputs[-1]

print("✅ Backbone Feature Maps:")
print(f"P3 shape: {p3.shape}")
print(f"P4 shape: {p4.shape}")
print(f"P5 shape: {p5.shape}")

# Inspect Detection Head configuration robustly
detect_layer = model.model.model[-1]

print("\n🔎 Detection Head Configuration:")
anchors = getattr(detect_layer, 'anchors', None)
print("Anchors:", anchors if anchors is not None else "Anchor-free")

strides = getattr(detect_layer, 'stride', None)
print("Strides:", strides.tolist() if strides is not None else "Not found")

nl = getattr(detect_layer, 'nl', len(strides) if strides is not None else "Unknown")
print("Number of detection layers (nl):", nl)

print(model)

✅ Backbone Feature Maps:
P3 shape: torch.Size([1, 128, 20, 20])
P4 shape: torch.Size([1, 384, 20, 20])
P5 shape: torch.Size([1, 256, 20, 20])

🔎 Detection Head Configuration:
Anchors: tensor([[ 0.5000,  1.5000,  2.5000,  ...,  8.5000,  9.5000, 10.5000],
        [ 0.5000,  0.5000,  0.5000,  ..., 20.5000, 20.5000, 20.5000]])
Strides: [8.0, 16.0, 32.0]
Number of detection layers (nl): 3
YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv

In [ ]:
# 2. Enhanced Custom Enhancement Modules
# ------------------------------------
print("\nPhase 2: Enhanced Custom Enhancement Modules")
print("==========================================")

class CBAM(nn.Module):
    def __init__(self, channels, reduction_ratio=8):  # Reduced ratio for stronger attention
        super().__init__()
        # Enhanced channel attention with both avg and max pooling
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.mlp = nn.Sequential(
            nn.Conv2d(channels, channels // reduction_ratio, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels // reduction_ratio, channels, 1, bias=False)
        )
        self.sigmoid_channel = nn.Sigmoid()

        # Spatial attention with larger kernel for better spatial context
        self.spatial_conv = nn.Conv2d(2, 1, kernel_size=7, padding=3, bias=False)
        self.sigmoid_spatial = nn.Sigmoid()

    def forward(self, x):
        # Channel attention with both avg and max pooling
        avg_out = self.mlp(self.avg_pool(x))
        max_out = self.mlp(self.max_pool(x))
        ca_weights = self.sigmoid_channel(avg_out + max_out)
        x_ca = ca_weights * x

        # Spatial attention
        avg_spatial = torch.mean(x_ca, dim=1, keepdim=True)
        max_spatial = torch.max(x_ca, dim=1, keepdim=True)[0]
        sa_input = torch.cat([avg_spatial, max_spatial], dim=1)
        sa_weights = self.sigmoid_spatial(self.spatial_conv(sa_input))
        x_sa = sa_weights * x_ca
        return x_sa

class TransformerEncoderBlock(nn.Module):
    def __init__(self, channels, num_heads=4, num_layers=2, dim_feedforward=None, dropout=0.1):
        super().__init__()
        if dim_feedforward is None:
            dim_feedforward = channels * 4  # Increased from 2x to 4x for better feature extraction

        self.channels = channels
        # Add positional embedding for better spatial awareness
        self.pos_embedding = nn.Parameter(torch.zeros(1, 256, channels))
        nn.init.trunc_normal_(self.pos_embedding, std=0.02)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=channels,
            nhead=num_heads,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True,
            activation=F.gelu  # Changed from ReLU to GELU for better gradient flow
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.norm = nn.LayerNorm(channels)

        # For QAT compatibility
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):
        # Save original for residual connection
        x_orig = x
        x = self.dequant(x)

        b, c, h, w = x.size()
        seq_len = h * w

        # Reshape and add positional embeddings
        seq = x.view(b, c, -1).permute(0, 2, 1)  # [B, HW, C]

        # Adapt positional embedding to the current sequence length
        if seq_len <= self.pos_embedding.shape[1]:
            pos_embed = self.pos_embedding[:, :seq_len, :]
        else:
            pos_embed = F.interpolate(
                self.pos_embedding.permute(0, 2, 1),
                size=seq_len,
                mode='linear'
            ).permute(0, 2, 1)

        seq = seq + pos_embed
        seq = self.norm(seq)  # Pre-norm for better stability

        # Apply transformer
        seq_out = self.transformer_encoder(seq)

        # Reshape back to spatial
        x_out = seq_out.permute(0, 2, 1).view(b, c, h, w)

        # Skip connection for stability
        x_out = x_out + x_orig

        # Requantize after Transformer
        x_out = self.quant(x_out)
        return x_out

class SmallObjectFeatures(nn.Module):
    def __init__(self, in_channels, feature_channels=128):
        super().__init__()
        # Expanded architecture for better small object detection
        self.conv_block = nn.Sequential(
            Conv(in_channels, feature_channels, k=3, p=1),
            Conv(feature_channels, feature_channels, k=3, p=1, d=2),  # Added dilation for larger receptive field
            Conv(feature_channels, feature_channels, k=3, p=1),
            Conv(feature_channels, feature_channels, k=3, p=1, s=1)   # Maintain resolution
        )

        # Add CBAM attention specifically for small objects
        self.attention = CBAM(feature_channels, reduction_ratio=4)  # More aggressive reduction for small objects

    def forward(self, x):
        x = self.conv_block(x)
        x = self.attention(x)
        return x

class BiFPN_FusionNode(nn.Module):
    def __init__(self, in_channels_high_res, in_channels_low_res, out_channels):
        super().__init__()
        # Enhanced with weighted fusion
        self.w1 = nn.Parameter(torch.ones(2, dtype=torch.float32), requires_grad=True)
        self.epsilon = 1e-4

        # Improved convolutions
        self.conv_high_res = Conv(in_channels_high_res, out_channels, k=1, p=0)
        self.conv_low_res = Conv(in_channels_low_res, out_channels, k=1, p=0)

        # Enhanced fusion with residual connection
        self.merge_conv1 = Conv(out_channels, out_channels, k=3, p=1)
        self.merge_conv2 = Conv(out_channels, out_channels, k=3, p=1)

    def forward(self, x_high_res, x_low_res):
        # Get normalized weights
        w = F.relu(self.w1)
        w = w / (torch.sum(w, dim=0) + self.epsilon)

        # Process features
        feat_high = self.conv_high_res(x_high_res)
        x_low_res_processed = self.conv_low_res(x_low_res)

        # Upsample if needed
        if x_low_res.shape[2:] != x_high_res.shape[2:]:
            x_low_res_upsampled = F.interpolate(
                x_low_res_processed,
                size=x_high_res.shape[2:],
                mode='bilinear',  # Changed from nearest to bilinear for smoother features
                align_corners=False
            )
        else:
            x_low_res_upsampled = x_low_res_processed

        # Weighted fusion
        fused = w[0] * feat_high + w[1] * x_low_res_upsampled

        # Enhanced processing with residual connection
        out = self.merge_conv1(fused)
        out = self.merge_conv2(out) + out  # Residual connection

        return out

# Enhanced DFL Loss for better distribution prediction
class EnhancedDFLLoss(nn.Module):
    def __init__(self, reg_max=16):
        super(EnhancedDFLLoss, self).__init__()
        self.reg_max = reg_max
        self.register_buffer('proj', torch.arange(0, reg_max).float())

    def forward(self, pred_dist, target):
        # Create target distribution using a Gaussian instead of one-hot
        b, a, c, h, w = pred_dist.shape
        pred_dist = pred_dist.view(b, a, c, -1)

        target_expanded = target.unsqueeze(-1)
        tl = torch.floor(target_expanded)
        tu = torch.ceil(target_expanded)

        # Create soft distribution using Gaussian around the target
        sigma = 0.5

        expanded_proj = self.proj.view(1, 1, -1, 1).expand(b, a, -1, h*w)
        gauss_weight_l = torch.exp(-((expanded_proj - tl) ** 2) / (2 * sigma ** 2))
        gauss_weight_u = torch.exp(-((expanded_proj - tu) ** 2) / (2 * sigma ** 2))

        # Normalize to sum to 1
        soft_targets = gauss_weight_l + gauss_weight_u
        soft_targets = soft_targets / (soft_targets.sum(dim=2, keepdim=True) + 1e-12)

        # Cross entropy loss on distributions
        loss = F.cross_entropy(
            pred_dist.permute(0, 1, 3, 2),
            soft_targets.permute(0, 1, 3, 2),
            reduction='none'
        )

        return loss.mean()

print("Enhanced custom modules defined.\n")


Phase 2: Enhanced Custom Enhancement Modules
Enhanced custom modules defined.



In [ ]:
# 3. Fixed Enhanced YOLO Model Structure
# ---------------------------------------------
print("\nPhase 3: Fixed Enhanced YOLO Model Structure")
print("======================================================")

class ModifiedYOLOBackbone(nn.Module):
    def __init__(self, original_model_sequential):
        super().__init__()
        self.layers = list(original_model_sequential.children())
        # Fixed tap points based on YOLOv12n structure
        self.p3_tap_idx = 4  # P3 features
        self.p4_tap_idx = 6  # P4 features
        self.p5_tap_idx = 8  # P5 features

        self.stage1 = nn.Sequential(*self.layers[0:self.p3_tap_idx+1])
        self.stage2 = nn.Sequential(*self.layers[self.p3_tap_idx+1:self.p4_tap_idx+1])
        self.stage3 = nn.Sequential(*self.layers[self.p4_tap_idx+1:self.p5_tap_idx+1])

        # Feature enhancement connections
        self.lateral_p5_p4 = Conv(256, 384, k=1)  # P5 to P4 lateral connection
        self.lateral_p4_p3 = Conv(384, 128, k=1)  # P4 to P3 lateral connection

        # CRITICAL: Add the f attribute that Ultralytics expects for export
        self.f = -1  # Add this to fix the export error

    def forward(self, x):
        # Process through backbone stages
        p3_feat = self.stage1(x)
        p4_feat = self.stage2(p3_feat)
        p5_feat = self.stage3(p4_feat)

        # FPN-like enhancement
        p5_up = F.interpolate(self.lateral_p5_p4(p5_feat), scale_factor=2, mode='bilinear', align_corners=False)
        p4_enhanced = p4_feat + p5_up

        p4_up = F.interpolate(self.lateral_p4_p3(p4_enhanced), scale_factor=2, mode='bilinear', align_corners=False)
        p3_enhanced = p3_feat + p4_up

        return p3_enhanced, p4_enhanced, p5_feat

class CustomEnhancedYOLOModel(nn.Module):
    def __init__(self, base_model_instance, nc):
        super().__init__()
        self.nc = nc
        original_sequential_model = base_model_instance.model.model

        # Enhanced backbone
        self.modified_backbone = ModifiedYOLOBackbone(original_sequential_model)

        # Channel dimensions
        self.ch_p3_tap = 128
        self.ch_p4_tap = 384
        self.ch_p5_tap = 256

        # Enhanced feature processing
        self.cbam_p3 = CBAM(self.ch_p3_tap)
        self.cbam_p4 = CBAM(self.ch_p4_tap)
        self.cbam_p5 = CBAM(self.ch_p5_tap)

        # Transformers for higher-level features
        self.transformer_p3 = TransformerEncoderBlock(self.ch_p3_tap, num_heads=2)
        self.transformer_p4 = TransformerEncoderBlock(self.ch_p4_tap, num_heads=4)
        self.transformer_p5 = TransformerEncoderBlock(self.ch_p5_tap, num_heads=4)

        # Increased channel dimensions for richer features
        self.neck_out_ch_p3 = 128  # Increased from 96
        self.neck_out_ch_p4 = 192  # Increased from 128
        self.neck_out_ch_p5 = 256  # Increased from 192

        # Feature fusion
        self.bifpn_p4_neck = BiFPN_FusionNode(self.neck_out_ch_p3, self.ch_p4_tap, self.neck_out_ch_p4)
        self.bifpn_p5_neck = BiFPN_FusionNode(self.neck_out_ch_p4, self.ch_p5_tap, self.neck_out_ch_p5)

        # YOLOv12 strides - keep as-is since YOLOv12 is anchor-free
        self.strides = torch.tensor([8.0, 16.0, 32.0])
        self.is_qat_enabled = False  # Default off, set True explicitly during training

        # Get the original detect head for reference
        base_detect = base_model_instance.model.model[-1]

        # Setup the detection head to match YOLOv12 design
        self.detect_head = Detect(
            nc=self.nc,
            ch=[self.neck_out_ch_p3, self.neck_out_ch_p4, self.neck_out_ch_p5]
        )

        # Ensure proper stride and other properties
        self.detect_head.stride = self.strides
        self.stride = self.strides  # For the overall model to have this attribute

        # If the detect head has a DFL component, preserve it
        if hasattr(base_detect, 'dfl'):
            self.detect_head.dfl = base_detect.dfl

        # Add enhanced DFL loss
        self.enhanced_dfl = EnhancedDFLLoss(reg_max=16)  # Match YOLOv12 reg_max

        # QAT support
        self.quant = QuantStub()
        self.dequant = DeQuantStub()

        # Make model subscriptable for export compatibility
        self.layers = nn.ModuleList([
            self.modified_backbone,
            self.cbam_p3, self.cbam_p4, self.cbam_p5,
            self.transformer_p3, self.transformer_p4, self.transformer_p5,
            self.detect_head
        ])

        # CRITICAL: Add the f attribute that Ultralytics expects for export
        self.f = -1  # Add this to fix the export error

    def fuse(self):
        print("Fusing modules in CustomEnhancedYOLOModel...")
        for m in self.modules():
            if type(m) is Conv and hasattr(m, 'bn') and hasattr(m, 'act'):
                 m.fuse_convs()
        print("Fusion complete.")
        return self

    def __getitem__(self, idx):
        return self.layers[idx]

    def forward(self, x):
        x = self.quant(x)

        # Get enhanced features from backbone
        p3_feat, p4_feat, p5_feat = self.modified_backbone(x)

        # Apply attention and transformer enhancements
        p3_enhanced = self.transformer_p3(self.cbam_p3(p3_feat))
        p4_enhanced = self.transformer_p4(self.cbam_p4(p4_feat))
        p5_enhanced = self.transformer_p5(self.cbam_p5(p5_feat))

        # Process P3 features
        p3_proj = nn.Sequential(
            nn.Conv2d(self.ch_p3_tap, self.neck_out_ch_p3, kernel_size=1, bias=False),
            nn.BatchNorm2d(self.neck_out_ch_p3),
            nn.SiLU(inplace=True)
        )(p3_enhanced)

        if self.training and self.is_qat_enabled:
            p3_proj = torch.fake_quantize_per_tensor_affine(
                p3_proj, scale=1.0/128.0, zero_point=0, quant_min=0, quant_max=255
            )

        # Process P4 features
        p4_proj = nn.Sequential(
            nn.Conv2d(self.ch_p4_tap, self.neck_out_ch_p4, kernel_size=1, bias=False),
            nn.BatchNorm2d(self.neck_out_ch_p4),
            nn.SiLU(inplace=True)
        )(p4_enhanced)

        if self.training and self.is_qat_enabled:
            p4_proj = torch.fake_quantize_per_tensor_affine(
                p4_proj, scale=1.0/128.0, zero_point=0, quant_min=0, quant_max=255
            )

        # Process P5 features
        p5_proj = nn.Sequential(
            nn.Conv2d(self.ch_p5_tap, self.neck_out_ch_p5, kernel_size=1, bias=False),
            nn.BatchNorm2d(self.neck_out_ch_p5),
            nn.SiLU(inplace=True)
        )(p5_enhanced)

        if self.training and self.is_qat_enabled:
            p5_proj = torch.fake_quantize_per_tensor_affine(
                p5_proj, scale=1.0/128.0, zero_point=0, quant_min=0, quant_max=255
            )

        # Improved feature fusion
        neck_p4 = self.bifpn_p4_neck(p4_proj, p3_proj)
        neck_p5 = self.bifpn_p5_neck(p5_proj, neck_p4)

        if self.training and self.is_qat_enabled:
            neck_p4 = torch.fake_quantize_per_tensor_affine(
                neck_p4, scale=1.0/128.0, zero_point=0, quant_min=0, quant_max=255
            )
            neck_p5 = torch.fake_quantize_per_tensor_affine(
                neck_p5, scale=1.0/128.0, zero_point=0, quant_min=0, quant_max=255
            )

        # Feed features to detection head (keep the order compatible with YOLOv12)
        features_for_head = [p3_proj, neck_p4, neck_p5]

        detections = self.detect_head(features_for_head)
        detections = self.dequant(detections)
        return detections

print("Enhanced anchor-free YOLOv12 model structure defined.\n")


Phase 3: Fixed Enhanced YOLO Model Structure
Enhanced anchor-free YOLOv12 model structure defined.



In [ ]:
# 5. Enhanced Main Script Logic
# ---------------------------------------------------------------------------------------
print("\nPhase 5: Enhanced Main Script Logic with Improved Training")
print("="*70)

# Paths (unchanged)
drive_base_path = '/content/drive/My Drive/'
dataset_root_in_drive = 'SemesterProjectDatas/CombinedData'
project_root_in_drive = 'SemesterProjectDatas/Model/Yolo12n/EnhancedYolo12n'

combined_dataset_path = os.path.join(drive_base_path, dataset_root_in_drive)
data_yaml = os.path.join(combined_dataset_path, 'data.yaml')
base_model_name = 'yolo12n.pt'
project_dir_enhanced_model = os.path.join(drive_base_path, project_root_in_drive)
experiment_name = 'run_yolo12n_enhanced_improved'

os.makedirs(project_dir_enhanced_model, exist_ok=True)

# Verify YAML
assert os.path.exists(data_yaml), f"ERROR: Dataset YAML file not found at '{data_yaml}'."

# Load dataset YAML
with open(data_yaml, 'r') as f:
    dataset_info = yaml.safe_load(f)
nc_from_yaml = int(dataset_info['nc'])
class_names_from_yaml = dataset_info['names']

# Initialize YOLO model
managed_model = YOLO(base_model_name)

# Instantiate enhanced YOLOv12n model
print("Initializing enhanced YOLOv12n model with improved architecture...")
custom_model_module = CustomEnhancedYOLOModel(
    base_model_instance=managed_model,
    nc=nc_from_yaml
)

# Assign model explicitly
managed_model.model.model = custom_model_module
managed_model.model.nc = nc_from_yaml
managed_model.model.names = dict(enumerate(class_names_from_yaml))

if hasattr(custom_model_module, 'stride'):
    managed_model.model.stride = custom_model_module.stride

# Two-stage training approach for better results
# Stage 1: Regular training without QAT
print("\nStage 1: Training without QAT (75 epochs)")
print("============================================")

# Enhanced training parameters
epochs_stage1 = 100
image_size = 640  # Increased from 640 for better detection
batch_size = 32   # Adjusted for larger image size
device_to_use = 'cuda' if torch.cuda.is_available() else 'cpu'

try:
    # First stage: Train without QAT
    results = managed_model.train(
        data=data_yaml,
        epochs=epochs_stage1,
        imgsz=image_size,
        batch=batch_size,
        project=project_dir_enhanced_model,
        name=experiment_name + "_stage1",
        exist_ok=True,
        device=device_to_use,
        patience=25,
        optimizer='AdamW',
        lr0=0.001,
        lrf=0.01,
        amp=True,
        dropout=0.1,  # Add some dropout for regularization
        mosaic=1.0,   # Full mosaic augmentation
        mixup=0.3,    # Add mixup
        degrees=15.0, # Increased rotation augmentation
        translate=0.2, # More translation
        copy_paste=0.3, # Add copy-paste for small objects
        # Customize loss weights to favor box precision
        box=7.5,  # Increased box loss weight
        cls=0.5,  # Standard classification weight
        dfl=1.5   # Standard DFL weight
    )
    print("✅ Stage 1 training completed successfully.")
    stage1_successful = True

except Exception as e:
    print(f"\n❌ ERROR DURING STAGE 1 TRAINING: {e}")
    import traceback
    traceback.print_exc()
    stage1_successful = False

# If first stage was successful, proceed to QAT stage
if stage1_successful:
    print("\nStage 2: QAT fine-tuning for 75 epochs")
    print("=======================================")

    # Load best model from stage 1
    best_model_path = os.path.join(project_dir_enhanced_model, experiment_name + "_stage1", 'weights', 'best.pt')
    if os.path.exists(best_model_path):
        # Load the best model from stage 1
        qat_model = YOLO(best_model_path)

        # Enable QAT in the model
        if hasattr(qat_model.model.model, 'is_qat_enabled'):
            qat_model.model.model.is_qat_enabled = True
            print("✅ QAT enabled for stage 2 training")

        try:
            # Continue training with QAT enabled
            results = qat_model.train(
                data=data_yaml,
                epochs=100,
                imgsz=image_size,
                batch=batch_size,
                project=project_dir_enhanced_model,
                name=experiment_name + "_stage2_qat",
                exist_ok=True,
                device=device_to_use,
                patience=25,
                optimizer='AdamW',
                lr0=0.0003,  # Lower learning rate for QAT stage
                lrf=0.001,
                amp=False,   # Disable mixed precision for QAT
                mosaic=0.8,  # Slightly reduce augmentation for fine-tuning
                mixup=0.1,
                degrees=7.0,
                translate=0.1,   # Reduced translation for fine-tuning
                copy_paste=0.1,  # Reduced copy-paste for QAT stage
                cache=True,      # Cache images for faster training
                box=7.5,         # Keep same loss weights for consistency
                cls=0.5,
                dfl=2.0          # Slightly increase DFL weight for better precision
            )
            print("✅ Stage 2 QAT training completed successfully.")
            qat_training_successful = True

        except Exception as e:
            print(f"\n❌ ERROR DURING STAGE 2 QAT TRAINING: {e}")
            import traceback
            traceback.print_exc()
            qat_training_successful = False

        # Post-training INT8 Conversion
        if qat_training_successful:
            best_qat_model_path = os.path.join(project_dir_enhanced_model, experiment_name + "_stage2_qat", 'weights', 'best.pt')

            if os.path.exists(best_qat_model_path):
                print("\nPerforming post-training INT8 conversion...")
                try:
                    # Load the best QAT model
                    base_model = YOLO(base_model_name)
                    quant_model = CustomEnhancedYOLOModel(
                        base_model_instance=base_model,
                        nc=nc_from_yaml
                    )

                    # Load weights correctly
                    loaded_weights = torch.load(best_qat_model_path)

                    # Check and extract state_dict properly
                    if 'model' in loaded_weights:
                        detection_model = loaded_weights['model']
                        model_state_dict = detection_model.state_dict()
                    else:
                        model_state_dict = loaded_weights

                    quant_model.load_state_dict(model_state_dict, strict=False)
                    quant_model.cpu().eval()

                    # Configure quantization for INT8 conversion
                    quant_model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

                    # Skip quantization for Transformer blocks
                    for module in quant_model.modules():
                        if isinstance(module, (TransformerEncoderBlock, CBAM)):
                            module.qconfig = None

                    # Prepare and convert the model
                    torch.quantization.prepare(quant_model, inplace=True)
                    quantized_model = torch.quantization.convert(quant_model)

                    # Save INT8 quantized model
                    quantized_model_path = os.path.join(project_dir_enhanced_model, experiment_name + "_stage2_qat", 'quantized_model_int8.pth')
                    torch.save(quantized_model.state_dict(), quantized_model_path)
                    print(f"✅ INT8 quantized model saved to: {quantized_model_path}")

                    # Export to TFLite for deployment
                    export_model = YOLO(base_model_name)
                    export_model.model.model = quantized_model
                    export_model.model.nc = nc_from_yaml
                    export_model.model.names = dict(enumerate(class_names_from_yaml))

                    # NOTE: Make sure the model has the 'f' attribute before exporting
                    if not hasattr(export_model.model.model, 'f'):
                        export_model.model.model.f = -1

                    tflite_path = os.path.join(project_dir_enhanced_model, experiment_name + "_stage2_qat", 'model_qat_int8.tflite')
                    try:
                        export_model.export(
                            format='tflite',
                            imgsz=image_size,
                            int8=True,
                            data=data_yaml,
                            device='cpu',
                            file=tflite_path
                        )
                        print(f"✅ TFLite QAT-INT8 model exported: {tflite_path}")
                    except Exception as e_export:
                        print(f"❌ Export error: {e_export}")
                        print("Trying alternative export approach...")
                        # Try simplified export
                        try:
                            export_model.export(format='onnx', imgsz=image_size, simplify=True)
                            print("✅ ONNX model exported as alternative")
                        except:
                            print("❌ Alternative export also failed")

                except Exception as e:
                    print(f"\n❌ ERROR DURING INT8 CONVERSION: {e}")
                    import traceback
                    traceback.print_exc()
            else:
                print(f"❌ Best QAT model weights not found: {best_qat_model_path}")
    else:
        print(f"❌ Best model from stage 1 not found: {best_model_path}")
else:
    print("\n❌ Stage 1 training failed. Skipping QAT and INT8 conversion.")


Phase 5: Enhanced Main Script Logic with Improved Training
Initializing enhanced YOLOv12n model with improved architecture...

Stage 1: Training without QAT (75 epochs)
Ultralytics 8.3.131 🚀 Python-3.11.12 torch-2.7.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.3, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/My Drive/SemesterProjectDatas/CombinedData/data.yaml, degrees=15.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.3, mode=train, mo

train: Scanning /content/drive/MyDrive/SemesterProjectDatas/CombinedData/train/labels.cache... 6527 images, 1141 backgrounds, 0 corrupt: 100%|██████████| 6527/6527 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 9723, len(boxes) = 30435. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 1.0±0.4 ms, read: 27.3±24.7 MB/s, size: 108.3 KB)


val: Scanning /content/drive/MyDrive/SemesterProjectDatas/CombinedData/valid/labels.cache... 591 images, 96 backgrounds, 0 corrupt: 100%|██████████| 591/591 [00:00<?, ?it/s]

...


50 epochs completed in 0.593 hours.
Optimizer stripped from /content/drive/My Drive/SemesterProjectDatas/Model/Yolo12n/EnhancedYolo12n/run_yolo12n_enhanced_improved_stage2_qat/weights/last.pt, 5.5MB
Optimizer stripped from /content/drive/My Drive/SemesterProjectDatas/Model/Yolo12n/EnhancedYolo12n/run_yolo12n_enhanced_improved_stage2_qat/weights/best.pt, 5.5MB

Validating /content/drive/My Drive/SemesterProjectDatas/Model/Yolo12n/EnhancedYolo12n/run_yolo12n_enhanced_improved_stage2_qat/weights/best.pt...
Ultralytics 8.3.131 🚀 Python-3.11.12 torch-2.7.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLOv12n summary (fused): 159 layers, 2,558,873 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:04<00:00,  2.38it/s]


                   all        591       2808      0.195      0.248      0.152     0.0826
                   Tin         43        100      0.253       0.28      0.212      0.148
             cardboard         69        102      0.164      0.186     0.0959     0.0555
       cigarette-butts         63        405      0.435     0.0399     0.0831     0.0316
food-wrappers-packaging        123        214      0.227      0.481      0.226      0.132
                litter        289       1640      0.222      0.345       0.19     0.0715
                  mask          1          1          0          0      0.249     0.0995
           paper-items         40        112       0.16      0.179     0.0824      0.039
          plastic-bags         25         32      0.107      0.375     0.0905     0.0479
plastic-bottles-containers         91        119      0.217      0.412      0.226      0.158
 small-packaging-trash         55         61      0.298      0.344      0.191      0.115
                

/usr/local/lib/python3.11/dist-packages/torch/ao/quantization/observer.py:244: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/ao/quantization/observer.py:1333: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


✅ INT8 quantized model saved to: /content/drive/My Drive/SemesterProjectDatas/Model/Yolo12n/EnhancedYolo12n/run_yolo12n_enhanced_improved_stage2_qat/quantized_model_int8.pth
❌ Export error: 'file' is not a valid YOLO argument. Similar arguments are i.e. ['profile=False'].

    Arguments received: ['yolo', '-f', '/root/.local/share/jupyter/runtime/kernel-de2dcf6d-9fe0-445c-8ca5-73a885959c28.json']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of frozenset({'segment', 'classify', 'obb', 'detect', 'pose'})
                MODE (required) is one of frozenset({'benchmark', 'track', 'train', 'predict', 'val', 'export'})
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 epochs with an initial learning_rate of 0.01
      

In [ ]:
# 6. Final Validation and Benchmarking
# ---------------------------------------------------------
print("\nPhase 6: Final Validation and Benchmarking")
print("=================================================")

try:
    # Identify the best available model
    if 'qat_training_successful' in locals() and qat_training_successful:
        final_model_path = os.path.join(project_dir_enhanced_model, experiment_name + "_stage2_qat", 'weights', 'best.pt')
        model_type = "QAT-enhanced"
    elif 'stage1_successful' in locals() and stage1_successful:
        final_model_path = os.path.join(project_dir_enhanced_model, experiment_name + "_stage1", 'weights', 'best.pt')
        model_type = "Enhanced (non-QAT)"
    else:
        raise Exception("No successfully trained models available")

    # Load final model
    print(f"Loading {model_type} model for final validation: {final_model_path}")
    final_model = YOLO(final_model_path)

    # Run comprehensive validation
    print("Running detailed validation with multiple metrics...")
    val_results = final_model.val(
        data=data_yaml,
        imgsz=image_size,
        batch=batch_size // 2,  # Smaller batch for validation
        plots=True,
        save_json=True,
        save_conf=True,     # Save confidence scores
        save_txt=True,      # Save labels
        verbose=True,
        conf=0.25,          # Lower confidence threshold to see more predictions
        iou=0.65            # Slightly higher IoU threshold for better precision
    )

    print(f"\n✅ Final {model_type} model validation results:")
    print(f"mAP50: {val_results.box.map50:.4f}")
    print(f"mAP50-95: {val_results.box.map:.4f}")
    print(f"Precision: {val_results.box.p:.4f}")
    print(f"Recall: {val_results.box.r:.4f}")

    # Create per-class performance visualization
    try:
        print("\nGenerating per-class performance visualization...")
        import matplotlib.pyplot as plt
        import pandas as pd
        import seaborn as sns

        # Extract per-class metrics
        classes = class_names_from_yaml
        ap_per_class = val_results.box.maps.tolist() if hasattr(val_results.box, 'maps') else [0] * len(classes)
        precision_per_class = val_results.box.p.tolist() if hasattr(val_results.box, 'p') else [0] * len(classes)
        recall_per_class = val_results.box.r.tolist() if hasattr(val_results.box, 'r') else [0] * len(classes)

        # Create dataframe for visualization
        metrics_df = pd.DataFrame({
            'Class': classes,
            'AP': ap_per_class,
            'Precision': precision_per_class,
            'Recall': recall_per_class
        })

        # Sort by AP
        metrics_df = metrics_df.sort_values('AP', ascending=False)

        # Create heatmap
        plt.figure(figsize=(14, 8))

        # Convert to wide format for heatmap
        heatmap_df = metrics_df.melt(id_vars=['Class'],
                                     value_vars=['AP', 'Precision', 'Recall'],
                                     var_name='Metric', value_name='Value')

        # Create pivoted dataframe for heatmap
        heatmap_pivot = heatmap_df.pivot(index='Class', columns='Metric', values='Value')

        # Plot heatmap
        sns.heatmap(heatmap_pivot, annot=True, cmap="YlGnBu", fmt=".3f", vmin=0, vmax=1)
        plt.title(f'Per-Class Performance Metrics - {model_type} Model', fontsize=16)
        plt.tight_layout()

        # Save figure
        class_metrics_path = os.path.join(project_dir_enhanced_model,
                             experiment_name + ("_stage2_qat" if 'qat_training_successful' in locals() and qat_training_successful else "_stage1"),
                             'class_performance_heatmap.png')
        plt.savefig(class_metrics_path, dpi=300, bbox_inches='tight')
        print(f"Per-class performance visualization saved to: {class_metrics_path}")

        # Also create a bar chart for easier comparison
        plt.figure(figsize=(15, 10))
        pos = np.arange(len(classes))
        width = 0.25

        plt.bar(pos - width, ap_per_class, width, alpha=0.7, color='#1f77b4', label='AP')
        plt.bar(pos, precision_per_class, width, alpha=0.7, color='#ff7f0e', label='Precision')
        plt.bar(pos + width, recall_per_class, width, alpha=0.7, color='#2ca02c', label='Recall')

        plt.ylabel('Score', fontsize=14)
        plt.title(f'Per-Class Performance Comparison - {model_type} Model', fontsize=16)
        plt.xticks(pos, classes, rotation=45, ha='right')
        plt.ylim(0, 1.0)
        plt.legend(loc='upper right')
        plt.grid(axis='y', alpha=0.3)

        # Save bar chart
        class_bar_path = os.path.join(project_dir_enhanced_model,
                          experiment_name + ("_stage2_qat" if 'qat_training_successful' in locals() and qat_training_successful else "_stage1"),
                          'class_performance_bars.png')
        plt.savefig(class_bar_path, dpi=300, bbox_inches='tight')
        print(f"Per-class bar chart saved to: {class_bar_path}")

    except Exception as e_viz:
        print(f"Error creating visualizations: {e_viz}")

    # Benchmark speed on different batch sizes and image sizes
    print("\nRunning inference speed benchmarks...")

    benchmark_results = []
    batch_sizes = [1, 4, 8]
    img_sizes = [640, image_size]

    for bs in batch_sizes:
        for imgsz in img_sizes:
            try:
                # Run benchmark test
                fps = final_model.predict(
                    source=None,  # Use random tensor for benchmarking
                    imgsz=imgsz,
                    batch=bs,
                    count=100,  # Number of frames to measure
                    device=device_to_use,
                    verbose=False
                )
                # Record result
                benchmark_results.append({
                    'Batch Size': bs,
                    'Image Size': imgsz,
                    'FPS': fps,
                    'Model Type': model_type
                })
                print(f"Batch size: {bs}, Image size: {imgsz}px - FPS: {fps:.1f}")
            except Exception as e_bench:
                print(f"Error during benchmark (BS={bs}, Size={imgsz}): {e_bench}")

    # Save benchmark results
    if benchmark_results:
        try:
            benchmark_df = pd.DataFrame(benchmark_results)
            benchmark_csv = os.path.join(project_dir_enhanced_model,
                            experiment_name + ("_stage2_qat" if 'qat_training_successful' in locals() and qat_training_successful else "_stage1"),
                            'benchmark_results.csv')
            benchmark_df.to_csv(benchmark_csv, index=False)
            print(f"Benchmark results saved to: {benchmark_csv}")

            # Create benchmark visualization
            plt.figure(figsize=(12, 8))
            for size in img_sizes:
                size_data = benchmark_df[benchmark_df['Image Size'] == size]
                plt.plot(size_data['Batch Size'], size_data['FPS'],
                         marker='o', linewidth=2, markersize=8,
                         label=f'Image Size: {size}px')

            plt.title(f'Inference Speed Benchmark - {model_type} Model', fontsize=16)
            plt.xlabel('Batch Size', fontsize=14)
            plt.ylabel('Frames Per Second (FPS)', fontsize=14)
            plt.xticks(batch_sizes)
            plt.grid(True, alpha=0.3)
            plt.legend(fontsize=12)

            benchmark_plot = os.path.join(project_dir_enhanced_model,
                             experiment_name + ("_stage2_qat" if 'qat_training_successful' in locals() and qat_training_successful else "_stage1"),
                             'benchmark_plot.png')
            plt.savefig(benchmark_plot, dpi=300)
            print(f"Benchmark visualization saved to: {benchmark_plot}")

        except Exception as e_bench_viz:
            print(f"Error saving benchmark results: {e_bench_viz}")

    # Comparison with baseline model
    print("\nComparing with baseline YOLOv12n model...")
    try:
        # Load baseline YOLOv12n
        baseline_model = YOLO('yolo12n.pt')

        # Validate baseline on same dataset
        baseline_results = baseline_model.val(
            data=data_yaml,
            imgsz=640,  # Original size for baseline
            batch=batch_size // 2,
            verbose=False
        )

        # Print comparison
        print("\n📊 Performance Comparison:")
        print(f"{'Metric':<12} {'Baseline':<10} {'Enhanced':<10} {'Improvement':<10}")
        print("-" * 42)

        baseline_map50 = baseline_results.box.map50 if hasattr(baseline_results.box, 'map50') else 0
        enhanced_map50 = val_results.box.map50 if hasattr(val_results.box, 'map50') else 0
        map50_improvement = enhanced_map50 - baseline_map50
        map50_percent = (map50_improvement / max(baseline_map50, 0.001)) * 100

        baseline_map = baseline_results.box.map if hasattr(baseline_results.box, 'map') else 0
        enhanced_map = val_results.box.map if hasattr(val_results.box, 'map') else 0
        map_improvement = enhanced_map - baseline_map
        map_percent = (map_improvement / max(baseline_map, 0.001)) * 100

        print(f"{'mAP50':<12} {baseline_map50:.4f}{'':<4} {enhanced_map50:.4f}{'':<4} {map50_improvement:.4f} ({map50_percent:+.1f}%)")
        print(f"{'mAP50-95':<12} {baseline_map:.4f}{'':<4} {enhanced_map:.4f}{'':<4} {map_improvement:.4f} ({map_percent:+.1f}%)")

        # Create comparison visualization
        plt.figure(figsize=(10, 6))
        models = ['Baseline YOLOv12n', f'Enhanced {model_type}']
        map50_values = [baseline_map50, enhanced_map50]
        map_values = [baseline_map, enhanced_map]

        x = np.arange(len(models))
        width = 0.35

        plt.bar(x - width/2, map50_values, width, label='mAP50', color='#1f77b4')
        plt.bar(x + width/2, map_values, width, label='mAP50-95', color='#ff7f0e')

        plt.ylabel('mAP Score', fontsize=14)
        plt.title('Performance Comparison', fontsize=16)
        plt.xticks(x, models)
        plt.ylim(0, max(max(map50_values), max(map_values)) * 1.2)

        # Add value labels on top of bars
        for i, v in enumerate(map50_values):
            plt.text(i - width/2, v + 0.01, f'{v:.4f}',
                     ha='center', fontsize=10, fontweight='bold')

        for i, v in enumerate(map_values):
            plt.text(i + width/2, v + 0.01, f'{v:.4f}',
                     ha='center', fontsize=10, fontweight='bold')

        plt.legend()
        plt.tight_layout()

        comparison_plot = os.path.join(project_dir_enhanced_model,
                           experiment_name + ("_stage2_qat" if 'qat_training_successful' in locals() and qat_training_successful else "_stage1"),
                           'baseline_comparison.png')
        plt.savefig(comparison_plot, dpi=300)
        print(f"Baseline comparison visualization saved to: {comparison_plot}")

    except Exception as e_compare:
        print(f"Error during baseline comparison: {e_compare}")

    print("\n✅ Enhanced YOLOv12n model validation and benchmarking complete!")

except Exception as e:
    print(f"\n❌ ERROR DURING FINAL VALIDATION: {e}")
    import traceback
    traceback.print_exc()


Phase 6: Final Validation and Benchmarking
Loading QAT-enhanced model for final validation: /content/drive/My Drive/SemesterProjectDatas/Model/Yolo12n/EnhancedYolo12n/run_yolo12n_enhanced_improved_stage2_qat/weights/best.pt
Running detailed validation with multiple metrics...
Ultralytics 8.3.131 🚀 Python-3.11.12 torch-2.7.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLOv12n summary (fused): 159 layers, 2,558,873 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.3±0.1 ms, read: 62.9±21.3 MB/s, size: 90.8 KB)


val: Scanning /content/drive/MyDrive/SemesterProjectDatas/CombinedData/valid/labels.cache... 591 images, 96 backgrounds, 0 corrupt: 100%|██████████| 591/591 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 937, len(boxes) = 2808. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:07<00:00,  5.14it/s]


                   all        591       2808       0.31      0.129      0.198      0.125
                   Tin         43        100      0.392        0.2      0.298      0.236
             cardboard         69        102      0.265     0.0882      0.158      0.108
       cigarette-butts         63        405        0.8    0.00988      0.405      0.182
food-wrappers-packaging        123        214      0.326      0.327      0.257      0.177
                litter        289       1640      0.422      0.186       0.28      0.129
                  mask          1          1          0          0          0          0
           paper-items         40        112      0.125     0.0179     0.0655     0.0458
          plastic-bags         25         32      0.125      0.125     0.0902      0.063
plastic-bottles-containers         91        119      0.377      0.244      0.275      0.207
 small-packaging-trash         55         61      0.478       0.18      0.291      0.186
                

Traceback (most recent call last):
  File "<ipython-input-15-6f15a4267334>", line 39, in <cell line: 0>
    print(f"Precision: {val_results.box.p:.4f}")
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: unsupported format string passed to numpy.ndarray.__format__


In [ ]:
# 7. Verify and Export INT8 Model
# ---------------------------------------------------------
print("\nPhase 7: Verify and Export INT8 Model for Deployment")
print("=================================================")

# Determine the best available INT8 model
int8_model_path = None

if 'qat_training_successful' in locals() and qat_training_successful:
    int8_model_path = os.path.join(project_dir_enhanced_model,
                     experiment_name + "_stage2_qat",
                     'quantized_model_int8.pth')
else:
    # Look for other INT8 models in the project directory
    for root, dirs, files in os.walk(project_dir_enhanced_model):
        for file in files:
            if file.endswith('int8.pth'):
                int8_model_path = os.path.join(root, file)
                break
        if int8_model_path:
            break

if int8_model_path and os.path.exists(int8_model_path):
    print(f"Found INT8 model at: {int8_model_path}")

    try:
        # Initialize base model and custom architecture
        print("Loading INT8 model for verification...")
        base_model = YOLO(base_model_name)

        # Create custom model with the right architecture
        int8_model = CustomEnhancedYOLOModel(
            base_model_instance=base_model,
            nc=nc_from_yaml
        )

        # Prepare for INT8
        int8_model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
        torch.quantization.prepare(int8_model, inplace=True)
        int8_model = torch.quantization.convert(int8_model, inplace=False)

        # Load the saved INT8 weights
        int8_model.load_state_dict(torch.load(int8_model_path))
        int8_model.eval()

        # Wrap in YOLO container for export
        export_model = YOLO(base_model_name)
        export_model.model.model = int8_model
        export_model.model.nc = nc_from_yaml
        export_model.model.names = dict(enumerate(class_names_from_yaml))

        # Ensure 'f' attribute is set for export
        if not hasattr(int8_model, 'f'):
            int8_model.f = -1

        # Try multiple export formats
        export_formats = ['tflite', 'onnx', 'torchscript']

        for format in export_formats:
            try:
                print(f"\nExporting to {format.upper()}...")
                if format == 'tflite':
                    export_path = export_model.export(
                        format=format,
                        imgsz=640,  # Use standard size for compatibility
                        int8=True,
                        data=data_yaml,
                        device='cpu'
                    )
                else:
                    export_path = export_model.export(
                        format=format,
                        imgsz=640,
                        device='cpu',
                        simplify=True
                    )
                print(f"✅ Successfully exported to {format.upper()}: {export_path}")

                # Create optimized export for edge devices if TFLite
                if format == 'tflite' and export_path:
                    try:
                        # For edge devices, create an optimized version
                        import tensorflow as tf

                        # Load the TFLite model
                        converter = tf.lite.TFLiteConverter.from_saved_model(export_path)

                        # Add optimization flags
                        converter.optimizations = [tf.lite.Optimize.DEFAULT]
                        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
                        converter.target_spec.supported_types = [tf.int8]
                        converter.inference_input_type = tf.uint8
                        converter.inference_output_type = tf.uint8

                        # Generate optimized model
                        optimized_model = converter.convert()

                        # Save the optimized model
                        optimized_path = export_path.replace('.tflite', '_optimized.tflite')
                        with open(optimized_path, 'wb') as f:
                            f.write(optimized_model)

                        print(f"✅ Created optimized TFLite model for edge devices: {optimized_path}")
                    except Exception as e_opt:
                        print(f"⚠️ Could not optimize TFLite model: {e_opt}")

            except Exception as e_exp:
                print(f"⚠️ Failed to export to {format}: {e_exp}")

        print("\n✅ INT8 model export process completed!")

    except Exception as e:
        print(f"❌ Error during INT8 model verification and export: {e}")
        import traceback
        traceback.print_exc()

else:
    print("❌ No INT8 model found. Run the training with QAT first.")


Phase 7: Verify and Export INT8 Model for Deployment
Found INT8 model at: /content/drive/My Drive/SemesterProjectDatas/Model/Yolo12n/EnhancedYolo12n/run_yolo12n_enhanced_improved_stage2_qat/quantized_model_int8.pth
Loading INT8 model for verification...


/usr/local/lib/python3.11/dist-packages/torch/ao/quantization/observer.py:244: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/ao/quantization/observer.py:1333: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


❌ Error during INT8 model verification and export: 'cbam_p3.mlp.0.bias'


/usr/local/lib/python3.11/dist-packages/torch/_utils.py:446: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,
Traceback (most recent call last):
  File "<ipython-input-16-11c330dc9919>", line 43, in <cell line: 0>
    int8_model.load_state_dict(torch.load(int8_model_path))
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 2573, in load_state_dict
    load(self, state_dict)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 2561, in load
    load(child, child_state_dict, child_prefix)  # noqa: F821
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 2561, in load
    load(child, child_state_d